In [59]:
import pandas as pd
import json
import re

In [60]:
# Load the CSV file
df = pd.read_csv("Sleep_health_and_lifestyle_dataset.csv")

In [61]:
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


1. remove sample values for Column Type: NUMERIC
   1. only keep NUMERIC and TEXT for now
2. replace space in column name with underscore
3. try to upload it with original csv together with jsonl and check if it works
4. add this as a new flag to GENERATE schema from csv
   1. database dialet Sqlite vs Postgres
   2. if sqlite, then use TEXT, if postgres, then use VARCHAR
   3. path to the csv, and output the schema(.jsonl)
   4. keep it a seperate file, schema_generator.py  

In [62]:
df.columns

Index(['Person ID', 'Gender', 'Age', 'Occupation', 'Sleep Duration',
       'Quality of Sleep', 'Physical Activity Level', 'Stress Level',
       'BMI Category', 'Blood Pressure', 'Heart Rate', 'Daily Steps',
       'Sleep Disorder'],
      dtype='object')

In [63]:
# Extract the schema information
schema = df.dtypes.to_dict()
schema

{'Person ID': dtype('int64'),
 'Gender': dtype('O'),
 'Age': dtype('int64'),
 'Occupation': dtype('O'),
 'Sleep Duration': dtype('float64'),
 'Quality of Sleep': dtype('int64'),
 'Physical Activity Level': dtype('int64'),
 'Stress Level': dtype('int64'),
 'BMI Category': dtype('O'),
 'Blood Pressure': dtype('O'),
 'Heart Rate': dtype('int64'),
 'Daily Steps': dtype('int64'),
 'Sleep Disorder': dtype('O')}

In [66]:
schema_list = []
for key, value in schema.items():
    if " " in key:
        new_key = key.replace(" ", "_") # Remove the space in the column name
    else:
        new_key = key
    if value == "object":
        value = "TEXT"
        unique_values = df[key].dropna().unique().tolist()
        print(f"unique_values: {unique_values[0]}")
        if not bool(re.search(r'[A-Za-z]', unique_values[0])):
            schema_list.append({"Column Name": new_key, "Column Type": value})
        else:
            schema_list.append({"Column Name": new_key, "Column Type": value, "Sample Values": unique_values})
    else: 
        value = "NUMERIC"
        schema_list.append({"Column Name": new_key, "Column Type": value})

unique_values: Male
unique_values: Software Engineer
unique_values: Overweight
unique_values: 126/83
unique_values: Sleep Apnea


[{'Column Name': 'Person_ID', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Gender',
  'Column Type': 'TEXT',
  'Sample Values': ['Male', 'Female']},
 {'Column Name': 'Age', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Occupation',
  'Column Type': 'TEXT',
  'Sample Values': ['Software Engineer',
   'Doctor',
   'Sales Representative',
   'Teacher',
   'Nurse',
   'Engineer',
   'Accountant',
   'Scientist',
   'Lawyer',
   'Salesperson',
   'Manager']},
 {'Column Name': 'Sleep_Duration', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Quality_of_Sleep', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Physical_Activity_Level', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Stress_Level', 'Column Type': 'NUMERIC'},
 {'Column Name': 'BMI_Category',
  'Column Type': 'TEXT',
  'Sample Values': ['Overweight', 'Normal', 'Obese', 'Normal Weight']},
 {'Column Name': 'Blood_Pressure', 'Column Type': 'TEXT'},
 {'Column Name': 'Heart_Rate', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Daily_Steps', 'Colum

In [67]:
# Save the schema information to a JSON file
with open("table_info_test.jsonl", "w") as f:
    for item in schema_list:
        json.dump(item, f)
        f.write("\n")